In [1]:
%matplotlib inline

In [2]:
from pylab import rcParams
rcParams['figure.figsize'] = 15,8

In [5]:
import scipy.stats
import analysis

In [6]:
data = analysis.get_full_dict()

In [7]:
x = analysis.pull_data(data, data.keys(), 'I use a wide array of commands.')
y = analysis.pull_data(data, data.keys(), 'I feel comfortable using the shell.')

In [8]:
correlation, pval = scipy.stats.pearsonr(x, y)